In [1]:
# !pip install regex

In [2]:
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
import plotly.express as px

import re
import regex
from collections import Counter

### Paso 1: Extraer los datos del chat grupal de WhatsApp
1.1. Entrar al chat grupal
<br>
1.2. Hacer click a los 3 puntos de la parte superior derecha 
<br>
1.3. Entrar a "Más" 
<br>
1.4. Entrar a "Exportar chat"
<br>
1.5. Entrar a "Sin archivos"
<br>
1.6. Enviar archivo txt por el medio que se considere
<br>
1.7. Colocar en la ruta donde se va a leer el archivo txt
<br>

### Paso 2: Detectar fecha y hora de los mensajes

In [3]:
# Patrón regex para identificar el comienzo de cada línea del txt con la fecha y la hora
# Nota: El formato del archivo txt depende del dispositivo donde se exporte y, por ende, la forma de detectar el patrón varía
def IniciaConFechaYHora(s):
    # Ejemplo: '21/2/2021 11:27 a. m. - ... '
    patron = '^([1-9]|1[0-9]|2[0-9]|3[0-1])(\/)([1-9]|1[0-2])(\/)(2[0-9][0-9][0-9]) ([0-9]+):([0-9][0-9]) (a.\xa0m.|p.\xa0m.) -'
    resultado = re.match(patron, s)  # Verificar si cada línea del txt hace match con el patrón de fecha y hora
    if resultado:
        return True
    return False

### Paso 3: Encontrar miembros del grupo que enviaron los mensajes

In [4]:
# Patrón para encontrar a los miembros del grupo dentro del txt
def EncontrarMiembro(s):
    patrones = [
        '([\w]+):',                                    # Nombre
        '([\w]+[\s]+[\w]+):',                    # Nombre + Apellido
        '([\w]+[\s]+[\w]+[\s]+[\w]+):',    # Nombre 1 + Nombre 2 + Apellido
        '([+]\d{2} \d{3} \d{3} \d{3}):',     # Número de teléfono (Peru)
        '([\w]+)[\u263a-\U0001f999]+:', # Nombre + Emoji            
    ]
    patron = '^' + '|'.join(patrones)     
    resultado = re.match(patron, s)  # Verificar si cada línea del txt hace match con el patrón de miembro
    if resultado:
        return True
    return False

### Paso 4: Obtener partes del chat grupal: Fecha, Hora, Miembro y Mensaje

In [5]:
# Separar las partes de cada línea del txt: Fecha, Hora, Miembro y Mensaje
def ObtenerPartes(linea):   
    # Ejemplo: '21/2/2021 11:27 a. m. - Sandro: Todos debemos aprender a analizar datos'
    splitLinea = linea.split(' - ') 
    FechaHora = splitLinea[0]                     # '21/2/2021 11:27 a. m.'
    splitFechaHora = FechaHora.split(' ')   
    Fecha = splitFechaHora[0]                    # '21/2/2021'
    Hora = ' '.join(splitFechaHora[1:])          # '11:27 a. m.'
    Mensaje = ' '.join(splitLinea[1:])             # 'Sandro: Todos debemos aprender a analizar datos'
    if EncontrarMiembro(Mensaje): 
        splitMensaje = Mensaje.split(': ')      
        Miembro = splitMensaje[0]               # 'Sandro' 
        Mensaje = ' '.join(splitMensaje[1:])    # 'Todos debemos aprender a analizar datos'
    else:
        Miembro = None
    return Fecha, Hora, Miembro, Mensaje

### Paso 5: Obtener el dataframe del chat grupal

In [6]:
# Leer el archivo txt descargado del chat de WhatsApp
RutaChat = 'Chat de WhatsApp con IISE PUCP 2022 🔥.txt'

# Lista para almacenar los datos (Fecha, Hora, Miembro, Mensaje) de cada línea del txt 
DatosLista = [] 
with open(RutaChat, encoding="utf-8") as fp:
    fp.readline() # Eliminar primera fila relacionada al cifrado de extremo a extremo
    VerificarMensaje = [] # Lista para verificar que no existan mensajes vacíos 
    Fecha, Hora, Miembro = None, None, None
    while True:
        linea = fp.readline() 
        if not linea: 
            break
        linea = linea.strip() 
        if IniciaConFechaYHora(linea): # Si cada línea del txt coincide con el patrón fecha y hora         
            if len(VerificarMensaje) > 0: 
                # Añadir un elemento lista que contendrá los datos a la lista 'DatosLista' 
                DatosLista.append([Fecha, Hora, Miembro, ' '.join(VerificarMensaje)]) 
            VerificarMensaje.clear() 
            Fecha, Hora, Miembro, Mensaje = ObtenerPartes(linea) # Obtener datos de cada línea del txt
            VerificarMensaje.append(Mensaje) 
        else:
            VerificarMensaje.append(linea)

# Convertir la lista con los datos a dataframe
df = pd.DataFrame(DatosLista, columns=['Fecha', 'Hora', 'Miembro', 'Mensaje']) 

# Cambiar la columna Fecha a formato datetime
df['Fecha'] = pd.to_datetime(df['Fecha'], format="%d/%m/%Y")

# Eliminar los posibles campos vacíos del dataframe
# y lo que no son mensajes como cambiar el asunto del grupo o agregar a alguien
df = df.dropna()

# Rester el índice
df.reset_index(drop=True, inplace=True)
df

,Fecha,Hora,Miembro,Mensaje
0,2022-03-17,10:58 p. m.,+51 998 606 279,Se eliminó este mensaje
1,2022-03-17,11:00 p. m.,Shaki IISE,*Bienvenidos a la familia IISE PUCP* 💙. Estamo...
2,2022-03-17,11:01 p. m.,+51 939 072 454,Esperando este mensaje
3,2022-03-17,11:01 p. m.,+51 973 616 721,<Multimedia omitido>
4,2022-03-17,11:02 p. m.,+51 956 415 614,Holaaa!!!
...,...,...,...,...
660,2022-03-23,12:32 p. m.,+51 929 001 178,<Multimedia omitido>
661,2022-03-23,12:34 p. m.,Jesús Cucho,Cracks 🚀💙!
662,2022-03-23,12:36 p. m.,+51 999 400 471,<Multimedia omitido>
663,2022-03-23,1:16 p. m.,Esmeralda,Muchas graciaaas a todos por los saludos!! Y f...


### Paso 6: Filtrar de acuerdo al rango de fechas requerido

In [7]:
fecha_inicio = "2022-03-17"
fecha_fin = "2022-03-31"

despues_fecha_inicio = df["Fecha"] >= fecha_inicio
antes_fecha_fin = df["Fecha"] <= fecha_fin
df = df.loc[despues_fecha_inicio & antes_fecha_fin]
df

,Fecha,Hora,Miembro,Mensaje
0,2022-03-17,10:58 p. m.,+51 998 606 279,Se eliminó este mensaje
1,2022-03-17,11:00 p. m.,Shaki IISE,*Bienvenidos a la familia IISE PUCP* 💙. Estamo...
2,2022-03-17,11:01 p. m.,+51 939 072 454,Esperando este mensaje
3,2022-03-17,11:01 p. m.,+51 973 616 721,<Multimedia omitido>
4,2022-03-17,11:02 p. m.,+51 956 415 614,Holaaa!!!
...,...,...,...,...
660,2022-03-23,12:32 p. m.,+51 929 001 178,<Multimedia omitido>
661,2022-03-23,12:34 p. m.,Jesús Cucho,Cracks 🚀💙!
662,2022-03-23,12:36 p. m.,+51 999 400 471,<Multimedia omitido>
663,2022-03-23,1:16 p. m.,Esmeralda,Muchas graciaaas a todos por los saludos!! Y f...


### Paso 7: Quedarse con los mensajes que contienen links (URLs)

In [8]:
df = df.loc[df['Mensaje'].str.contains("https://", case=False)].reset_index(drop=True)
df

,Fecha,Hora,Miembro,Mensaje
0,2022-03-17,11:11 p. m.,Carlos Tarrillo,Holas soy Carlos 😊 coordinador de MKT 🥳💙 Un al...
1,2022-03-17,11:11 p. m.,Mateo Analytics,"Hola! Bienvenidos! Soy Mateo Anaya , *de Marke..."
2,2022-03-17,11:12 p. m.,Bresia,Hoaa soy Bresia 😌 bienvenid@s a tod@s! Soy coo...
3,2022-03-17,11:12 p. m.,+51 998 606 279,Yo soy Steph🙈 y estas son mis redes sociales: ...
4,2022-03-17,11:13 p. m.,+51 954 783 601,Hola Soy Sam y aquí dejo mis redes (solo uso m...
...,...,...,...,...
114,2022-03-21,11:30 p. m.,Pedro Jáuregui IISE,Resultado del Sorteo de Networking https://www...
115,2022-03-22,11:09 a. m.,Bresia,🥳🥳 ¡Atención! 📢📢 Se acaba de compartir el pos...
116,2022-03-22,1:23 p. m.,+51 963 978 191,💙📣 *Membresía IISE PUCP* 📣💙 Hola chic@s! Les c...
117,2022-03-22,2:39 p. m.,Bresia,Buenos tardes chicxs 😊 Se acaba de compartir ...


### Paso 8: Obtener links de los mensajes del chat grupal (en este caso redes sociales) 

In [9]:
for i in range(len(df)):
    mensaje =df.loc[i,'Mensaje']
    links = re.findall("(?P<url>https?://[^\s]+)", mensaje)#.group("url")
    for link in links:
        if 'instagram' in link and 'invites/contact' not in link: # Se añade evitar los links que contienen 'invites' porque existen perfiles que se comparten de la forma
            df.at[i,'Instagram'] = link                           # 'https://www.instagram.com/invites/contact/?i=1m1m3zmrmzqw6&utm_content=3jpjiin'
                                                                  # Curiosamente, se puede acceder de celular, pero no de computador (donde el bot se ejecuta)
        if 'facebook' in link:
            df.at[i,'Facebook'] = link 
            
        if 'linkedin' in link:
            df.at[i,'LinkedIn'] = link 

df = df.dropna(subset=['Instagram', 'Facebook', 'LinkedIn'], how='all').reset_index(drop=True)
df

,Fecha,Hora,Miembro,Mensaje,Instagram,Facebook,LinkedIn
0,2022-03-17,11:11 p. m.,Carlos Tarrillo,Holas soy Carlos 😊 coordinador de MKT 🥳💙 Un al...,https://www.instagram.com/carlostr.27/,https://www.facebook.com/carlos.cmtar.tarrillo/,https://www.linkedin.com/in/carlostarrillo/
1,2022-03-17,11:11 p. m.,Mateo Analytics,"Hola! Bienvenidos! Soy Mateo Anaya , *de Marke...",https://www.instagram.com/mateo_ap/,https://www.facebook.com/MateoAnayaP,https://www.linkedin.com/in/mateo-anaya-pardo
2,2022-03-17,11:12 p. m.,Bresia,Hoaa soy Bresia 😌 bienvenid@s a tod@s! Soy coo...,https://www.instagram.com/bresia1701/,https://www.facebook.com/bresia.stefany,https://www.linkedin.com/in/bresia-canchanya/
3,2022-03-17,11:12 p. m.,+51 998 606 279,Yo soy Steph🙈 y estas son mis redes sociales: ...,https://www.instagram.com/stephcoronelarenas/,https://www.facebook.com/stephanieeva.coronela...,https://www.linkedin.com/in/stefany-coronel-ar...
4,2022-03-17,11:13 p. m.,+51 954 783 601,Hola Soy Sam y aquí dejo mis redes (solo uso m...,https://instagram.com/puk_student.peru?utm_med...,NaN,NaN
...,...,...,...,...,...,...,...
103,2022-03-19,12:01 p. m.,+51 950 985 390,Hola a todo el mundo 🤗!!! Me llamo Brenda Llam...,https://www.instagram.com/brend_lg_25/,https://www.facebook.com/brendalg.gatto/,https://www.linkedin.com/in/brenda-liz-llamocc...
104,2022-03-19,1:51 p. m.,+51 995 544 909,"Hola a todos/as/es, soy María José Melendez pe...",https://instagram.com/majomelendez29?utm_mediu...,https://www.facebook.com/Majo.Melendez.Manchego,https://pe.linkedin.com/in/maria-jose-melendez...
105,2022-03-19,1:59 p. m.,+51 995 544 909,https://pe.linkedin.com/in/maria-jose-melendez...,NaN,NaN,https://pe.linkedin.com/in/maria-jose-melendez...
106,2022-03-21,10:22 p. m.,Pedro Jáuregui IISE,"Chicos por si quieren ver la grabación, tienen...",NaN,https://www.facebook.com/groups/624530571538373,NaN


### Paso 9: Guardar la tabla

In [10]:
df.to_csv('links_de_redes_sociales.csv', index=False)